# Saliency Map Demo

This demo is divided into two parts -- model training and . The first is a
slightly simplified example from the `torchgeo` documentation, originally
written by Caleb Robinson, which fine-tunes a land cover classifier using a
ResNet classifier pretrained on a large satellite imagery dataset. The second
shows how we can use the captum package to compute gradient and integrated
gradient-based saliency maps to explain individual predictions, with an eye
especially towards the model's mistakes.

To run this notebook yourself, you can setup a new conda environment with the packages installed:

In [ ]:
# select "stat992_week2" as your notebook kernel
!conda env create -f https://github.com/krisrs1128/stat992_s25/raw/refs/heads/main/demos/stat992_week2.yml

## Model Training

_Written by: Caleb Robinson_

In this tutorial, we demonstrate TorchGeo trainers to train and test a model. We will use the [EuroSAT](https://torchgeo.readthedocs.io/en/stable/api/datasets.html#eurosat) dataset throughout this tutorial. Specifically, a subset containing only 100 images. We will train models to predict land cover classes.

It's recommended to run this notebook on Google Colab if you don't have your own GPU. Click the "Open in Colab" button above to get started.

In [ ]:
%matplotlib inline
%load_ext tensorboard

import torch
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger

from torchgeo.datamodules import EuroSAT100DataModule
from torchgeo.models import ResNet18_Weights
from torchgeo.trainers import ClassificationTask
from captum.attr import Saliency, IntegratedGradients
from captum.attr import visualization as viz
torch.manual_seed(20250130)

## Lightning modules

Our trainers use [Lightning](https://lightning.ai/docs/pytorch/stable/) to organize both the training code, and the dataloader setup code. This makes it easy to create and share reproducible experiments and results.

First we'll create a `EuroSAT100DataModule` object which is simply a wrapper around the [EuroSAT100](https://torchgeo.readthedocs.io/en/latest/api/datasets.html#eurosat) dataset. This object 1.) ensures that the data is downloaded, 2.) sets up PyTorch `DataLoader` objects for the train, validation, and test splits, and 3.) ensures that data from the same region **is not** shared between the training and validation sets so that you can properly evaluate the generalization performance of your model.

The following variables can be modified to control training.

In [4]:
batch_size = 25
num_workers = 2
max_epochs = 50

In [5]:
datamodule = EuroSAT100DataModule(root='.', batch_size=batch_size, num_workers=num_workers, download=True)

Next, we create a `ClassificationTask` object that holds the model object, optimizer object, and training logic. We will use a ResNet-18 model that has been pre-trained on Sentinel-2 imagery.

In [6]:
task = ClassificationTask(
    loss='ce',
    model='resnet18',
    weights=ResNet18_Weights.SENTINEL2_ALL_MOCO,
    in_channels=13,
    num_classes=10,
    lr=0.05,
    patience=5,
)

## Training

Now that we have the Lightning modules set up, we can use a Lightning [Trainer](https://lightning.ai/docs/pytorch/stable/common/trainer.html) to run the training and evaluation loops. There are many useful pieces of configuration that can be set in the `Trainer` -- below we set up model checkpointing based on the validation loss, early stopping based on the validation loss, and a TensorBoard based logger. We encourage you to see the [Lightning docs](https://lightning.ai/docs/pytorch/stable/) for other options that can be set here, e.g. CSV logging, automatically selecting your optimizer's learning rate, and easy multi-GPU training.

In [7]:
accelerator = 'mps' if torch.backends.mps.is_available() else 'cpu'
logger = TensorBoardLogger(save_dir='.')

For tutorial purposes we deliberately lower the maximum number of training epochs.

In [ ]:
trainer = Trainer(
    accelerator=accelerator,
    log_every_n_steps=1,
    logger=logger,
    min_epochs=1,
    max_epochs=max_epochs,
)

When we first call `.fit(...)` the dataset will be downloaded and checksummed (if it hasn't already). After this, the training process will kick off, and results will be saved so that TensorBoard can read them.

In [ ]:
trainer.fit(model=task, datamodule=datamodule)

We launch TensorBoard to visualize various performance metrics across training and validation epochs. We can see that our model is just starting to converge, and would probably benefit from additional training time and a lower initial learning rate.

In [ ]:
%tensorboard --logdir .

## Interpretability

First, let's extract a batch of examples and compute predictions on them.

In [11]:
data_iter = iter(datamodule.train_dataloader())
batch = next(data_iter)

images = batch["image"]
labels = batch["label"]
predicted = task(batch["image"]).argmax(dim=1)

Let's now visualize all the predictions for this batch along with the ground truth class.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

label_ = ["Annual Crop", "Forest", "Herbaceous Vegetation", "Highway",
          "Industrial Buildings", "Pasture", "Permanent Crop", 
          "Residential Buildings", "River", "Sea & Lake"]

for index in range(batch_size):
    input_image = images[index].unsqueeze(0)
    x = input_image.detach().numpy()[0, [3,2,1]]

    # plot the model's prediction
    plt.imshow(np.transpose(x, (1, 2, 0))/ x.max())
    plt.show()
    output = task(input_image)

    print(index)
    print("Predicted: ", label_[predicted[index]])
    print("True: ", label_[labels[index]])

Some of the mistakes seem to occur on ambiguous regions, where parts of the image seem to correspond to one class while other parts correspond to a different one. We can try to gauge this hypothesis by using different types of saliency maps. The results are not definitive. For both saliency maps and integrated gradients, the model seems to attend to parts of the image from both the correct class and the distracting background. Our hypothesis might be part of the story, but it doesn't seem to be the complete one.

In [ ]:
index = 3
input_image = batch["image"][index].unsqueeze(0)
x = input_image.detach().numpy()[0, [3,2,1]]
x = np.transpose(x, (1, 2, 0))

# initialize a Saliency object
saliency = Saliency(task.model)
z = saliency.attribute(input_image, target=predicted[index])

# postprocess and visualize
z = z.detach().numpy()[0]
z = np.transpose(z, (1, 2, 0))
_ = viz.visualize_image_attr(z, x, method = "blended_heat_map")

In [ ]:
ig =  # ?

# postprocess the integrated gradients
#z = z.detach().numpy()[0]
#z = np.transpose(z, (1, 2, 0))